In [1]:
import numpy as np
import pandas as pd

## 一、分组模式及对象

### 1.分组的一般模式

分组操作在日常生活中使用极其广泛，例如：

依据 **性别** 分组，统计全国人口 寿命 的 平均值

依据 **季节** 分组，对每一个季节的 温度 进行 组内标准化

依据 **班级** 分组，筛选出组内 数学分数 的 平均值超过80分的班级

从上述的几个例子中不难看出，想要实现分组操作，**必须明确三个要素**：**分组依据** 、 **数据来源** 、 **操作及其返回结果** 。同时从充分性的角度来说，如果明确了这三方面，就能确定一个分组操作，从而分组代码的一般模式即：

```python
df.groupby(by='分组依据')[数据来源].操作()
```
例如第一个例子中的代码就应该如下：
```python
df.groupby('Gender')['Longevity'].mean()
```


现在返回到学生体测的数据集上，如果想要**按照性别统计身高中位数**，就可以如下写出：

In [2]:
df = pd.read_csv('./data/learn_pandas.csv')
# 分组依据： Gender
# 数据来源：Height
# 操作：median

df.groupby('Gender')['Height'].median()

# Gender
# Female    159.6
# Male      173.4
# Name: Height, dtype: float64

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

### 2. 分组依据的本质
前面提到的若干例子都是以单一维度进行分组的，比如根据性别，如果现在需要根据多个维度进行分组，该如何做？事实上，只需在 groupby 中传入相应列名构成的列表即可。例如，**现希望根据学校和性别进行分组**，统计身高的均值就可以如下写出：

In [3]:
df.groupby(['School','Gender'])['Height'].mean()

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

目前为止， groupby 的分组依据都是直接可以从列中按照名字获取的，那如果希望通过一定的**复杂逻辑**来分组，例如**根据学生体重是否超过总体均值来分组**，同样还是计算身高的均值。

首先应该先写出分组条件：

In [4]:
condition = df['Weight'] > df['Weight'].mean()

将其传入groupby中：

In [5]:
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

#### 练一练：
请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值。

In [6]:
# 使用上下四分位数进行标记数据
# df['Weight_biaoji'] = df['Weight'].mask(df['Weight'] > df['Weight'].quantile(0.75),'high')\
res = df['Weight'].mask(df['Weight'] > df['Weight'].quantile(0.75),'high')\
.mask(df['Weight'] < df['Weight'].quantile(0.25),'low')\
    .mask((df['Weight'] >= df['Weight'].quantile(0.25) ) & (df['Weight'] <= df['Weight'].quantile(0.75)),'normal' ) 

# df.groupby('Weight_biaoji')['Height'].mean()
df.groupby(res)['Height'].mean()

Weight
high      174.935714
low       153.753659
normal    161.883516
Name: Height, dtype: float64

从索引可以看出，其实最后产生的结果就是按照条件列表中元素的值（此处是 True 和 False ）来分组，下面用随机传入字母序列来验证这一想法：

In [7]:
# 随机传入字母列表来分组

item = np.random.choice(list('abc'),df.shape[0])
# 按照分组结果计算对应的身高平均值
df.groupby(item)['Height'].mean()


a    162.784286
b    163.762712
c    163.185185
Name: Height, dtype: float64

此处的索引就是原先item中的元素，**如果传入多个序列进入 groupby ，那么最后分组的依据就是这两个序列对应行的唯一组合**：

In [8]:
df.groupby([condition,item])['Height'].mean()

Weight   
False   a    159.243396
        b    159.228205
        c    158.502857
True    a    173.823529
        b    172.605000
        c    171.810526
Name: Height, dtype: float64

由此可以看出，之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，**最后分组的依据来自于数据来源组合的unique值**，通过 drop_duplicates 就能知道具体的组类别：

In [9]:
# 只对School和Gender列去重，不包含其他列
df[['School','Gender']].drop_duplicates()

# 对School和Gender列去重，包含其他列
# df.drop_duplicates(['School','Gender'],keep='first')


# 返回的是序列Series

# 下面两种方法是等价的
df.groupby([df['School'], df['Gender']])['Height'].mean()
df.groupby(['School','Gender'])['Height'].mean()
# type(df.groupby(['School','Gender'])['Height'].mean())

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

### 3. Groupby对象
能够注意到，最终具体做分组操作时，所调用的方法都来自于 pandas 中的 groupby 对象，这个对象上定义了许多方法，也具有一些方便的属性。

In [10]:
gb = df.groupby(['School','Grade'])
gb
# <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001DBEC772050>

通过 `ngroups` 属性，可以得到分组个数：

In [11]:
gb.ngroups
# 16

16

通过 `groups` 属性，可以返回从 **组名** 映射到 **组索引列表** 的字典：

In [12]:
res = gb.groups

# 字典的值由于是索引，元素个数过多，此处只展示字典的键
res.keys()

dict_keys([('A', 'Freshman'), ('A', 'Junior'), ('A', 'Senior'), ('A', 'Sophomore'), ('B', 'Freshman'), ('B', 'Junior'), ('B', 'Senior'), ('B', 'Sophomore'), ('C', 'Freshman'), ('C', 'Junior'), ('C', 'Senior'), ('C', 'Sophomore'), ('D', 'Freshman'), ('D', 'Junior'), ('D', 'Senior'), ('D', 'Sophomore')])

上一小节介绍了可以通过 `drop_duplicates` 得到具体的组类别，现请用 `groups` 属性完成类似的功能。

In [13]:
df.groupby(['School','Gender']).groups.keys()

dict_keys([('A', 'Female'), ('A', 'Male'), ('B', 'Female'), ('B', 'Male'), ('C', 'Female'), ('C', 'Male'), ('D', 'Female'), ('D', 'Male')])

当 `size` 作为 `DataFrame` 的属性时，返回的是**表长乘以表宽的大小**，但在 groupby 对象上表示**统计每个组的元素个数**：

In [14]:
gb.size()

School  Grade    
A       Freshman     13
        Junior       17
        Senior       22
        Sophomore     5
B       Freshman     13
        Junior        8
        Senior        8
        Sophomore     5
C       Freshman      9
        Junior       12
        Senior       11
        Sophomore     8
D       Freshman     17
        Junior       22
        Senior       14
        Sophomore    16
dtype: int64

通过 `get_group` 方法可以直接获取所在组对应的行，此时必须知道组的具体名字：

In [15]:
# DataFrame对象
gb.get_group(('A','Junior')).iloc[:3,:3]

,School,Grade,Name
31,A,Junior,Feng Zheng
42,A,Junior,Mei Zhang
50,A,Junior,Xiaoli Wang


这里列出了2个属性和2个方法，而先前的 mean 、 median 都是 groupby 对象上的方法，这些函数和许多其他函数的操作具有高度相似性，将在之后的小节进行专门介绍。

### 4. 分组的三大操作
熟悉了一些分组的基本知识后，重新回到开头举的三个例子，可能会发现一些端倪，即这三种类型分组返回的数据型态并不一样：

第一个例子中，每一个组返回一个标量值，可以是平均值、中位数、组容量 size 等

第二个例子中，做了原序列的标准化处理，也就是说每组返回的是一个 Series 类型

第三个例子中，既不是标量也不是序列，返回的整个组所在行的本身，即返回了 DataFrame 类型

由此，引申出分组的三大操作：**聚合、变换和过滤**，分别对应了三个例子的操作，下面就要分别介绍相应的 agg 、 transform 和 filter 函数及其操作。

## 二、聚合函数
### 1. 内置聚合函数
在介绍agg之前，首先要了解一些直接定义在groupby对象的聚合函数，因为它的速度基本都会经过内部的优化，使用功能时应当优先考虑。根据返回标量值的原则，包括如下函数： 
max/min/mean/median/count/all/any/idxmax/idxmin

mad/nunique/skew/quantile/sum/std/var/sem/size/prod 。

In [16]:
gb = df.groupby('Gender')['Height']

# 每个组的最小值的索引
gb.idxmin()
# Gender
# Female    143
# Male      199
# Name: Height, dtype: int64

gb.quantile(0.95)
# Gender
# Female    166.8
# Male      185.9
# Name: Height, dtype: float64

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

#### 练一练：
请查阅文档，明确 all/any/mad/skew/sem/prod 函数的含义。

##### 总结表格
| 函数     | 核心功能                 | 关键参数         | 适用场景                     |
| -------- | ------------------------ | ---------------- | ---------------------------- |
| `all()`  | 检查是否所有元素为真值   | `axis`、`skipna` | 验证条件是否全满足           |
| `any()`  | 检查是否存在至少一个真值 | `axis`、`skipna` | 验证是否存在符合条件的元素   |
| `mad()`(已被弃用)  | 计算平均绝对偏差         | `axis`、`skipna` | 衡量数据离散程度（抗异常值） |
| `skew()` | 计算分布偏度(衡量数据分布的不对称程度)             | `axis`、`skipna` | 分析数据分布对称性           |
| `sem()`  | 计算均值标准误(sem = std(x) / sqrt(n))           | `axis`、`ddof`   | 评估均值的可靠性             |
| `prod()` | 计算元素乘积             | `axis`、`skipna` | 累计乘积计算   


In [17]:
# 偏度
gb.skew()
# Gender
# Female   -0.219253
# Male      0.437535
# Name: Height, dtype: float64

# 均值标准误
gb.sem()
# Gender
# Female    0.439893
# Male      0.986985
# Name: Height, dtype: float64

# 元素乘积
gb.prod()
# Gender
# Female    4.232080e+290
# Male      1.594210e+114
# Name: Height, dtype: float64


Gender
Female    4.232080e+290
Male      1.594210e+114
Name: Height, dtype: float64

这些聚合函数当传入的数据来源包含多个列时，将**按照列进行迭代计算**：

In [18]:
gb = df.groupby('Gender')[['Height','Weight']]

gb.max()


,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


### 2. agg方法
虽然在 groupby 对象上定义了许多方便的函数，但仍然有以下不便之处：

- 无法同时使用多个函数

- 无法对特定的列使用特定的聚合函数

- 无法使用自定义的聚合函数

- 无法直接对结果的列名在聚合前进行自定义命名

下面说明如何通过 agg 函数解决这四类问题：

#### 【a】使用多个函数

当使用多个聚合函数时，**需要用列表的形式把内置聚合函数对应的字符串传入**，先前提到的所有字符串都是合法的。

In [19]:
gb.agg(['sum','mean','max','idxmax','skew'])

Height                                     Weight                   \
            sum       mean    max idxmax      skew     sum       mean   max   
Gender                                                                        
Female  21014.0  159.19697  170.2     28 -0.219253  6469.0  47.918519  63.0   
Male     8854.9  173.62549  193.9    193  0.437535  3929.0  72.759259  89.0   

                         
       idxmax      skew  
Gender                   
Female     28 -0.268482  
Male        2 -0.332393

从结果看，此时的列索引为多级索引，第一层为数据源，第二层为使用的聚合方法，分别逐一对列使用聚合，因此结果为6列。

#### 【b】对特定的列使用特定的聚合函数

对于方法和列的特殊对应，可以通过构造字典传入 agg 中实现，**其中字典以列名为键，以聚合字符串或字符串列表为值**。

In [20]:
gb.agg({'Height':['mean','max'],'Weight':'count'})

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

##### 练一练：

请使用【b】中的传入字典的方法完成【a】中等价的聚合任务。



In [21]:
gb.agg({'Height':['sum','mean','max','idxmax','skew'],'Weight':['sum','mean','max','idxmax','skew']})

Height                                     Weight                   \
            sum       mean    max idxmax      skew     sum       mean   max   
Gender                                                                        
Female  21014.0  159.19697  170.2     28 -0.219253  6469.0  47.918519  63.0   
Male     8854.9  173.62549  193.9    193  0.437535  3929.0  72.759259  89.0   

                         
       idxmax      skew  
Gender                   
Female     28 -0.268482  
Male        2 -0.332393

##### 【c】使用自定义函数

在 agg 中可以使用具体的自定义函数， 需要**注意传入函数的参数是之前数据源中的列，逐列进行计算** 。下面分组计算身高和体重的极差：

In [22]:
gb.agg(lambda x:x.max()-x.min())

,Height,Weight
Gender,,
Female,24.8,29.0
Male,38.2,38.0


##### 练一练
在 groupby 对象中可以使用 describe 方法进行统计信息汇总，请同时使用多个聚合函数，完成与该方法相同的功能。

In [23]:
gb.describe()

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

In [24]:
gb.agg({'Height':['count','mean','std','min', ('q25', lambda x: x.quantile(0.25)), ('q25', lambda x: x.quantile(0.5)), ('q25', lambda x: x.quantile(0.75)),'max'],'Weight':['count','mean','std','min', ('q25', lambda x: x.quantile(0.25)), ('q25', lambda x: x.quantile(0.5)), ('q25', lambda x: x.quantile(0.75)),'max']})

Height                                                              \
        count       mean       std    min      q25    q25      q25    max   
Gender                                                                      
Female    132  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male       51  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   q25   q25    q25   max  
Gender                                                             
Female    135  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male       54  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

In [25]:
def my_func(s):
    # 接收一个 Series 对象 s（代表分组后的某一列数据）
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    # s.mean()：计算当前分组的均值（分组内的平均值）。
    # df[s.name].mean()：
    # s.name 是当前列的列名（例如，若处理 'Height' 列，s.name 就是 'Height'）。
    # df[s.name].mean() 计算整个 DataFrame 中该列的总均值（不分组的整体平均值）。
    return res

gb.agg(my_func)


,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


#### 【d】聚合结果重命名

如果想要对聚合结果的列名进行**重命名，只需要将上述函数的位置改写成元组，元组的第一个元素为新的名字，第二个位置为原来的函数**，包括聚合字符串和自定义函数，现举若干例子说明：

In [26]:
gb.agg({'Height':[('myfunc',my_func),'sum'],'Weight':lambda x:x.max()})

Height            Weight
       myfunc      sum <lambda>
Gender                         
Female    Low  21014.0     63.0
Male     High   8854.9     89.0

另外需要**注意，使用对一个或者多个列使用单个聚合的时候，重命名需要加方括号**，否则就不知道是新的名字还是手误输错的内置函数字符串：

In [27]:
# 计算所有columns的sum指标,重命名为my_sum
gb.agg([('my_sum','sum')])

,Height,Weight
,my_sum,my_sum
Gender,,
Female,21014.0,6469.0
Male,8854.9,3929.0


In [28]:
gb.agg({'Height': [('my_func', my_func), 'sum'],
        'Weight': [('range', lambda x:x.max())]})


Height          Weight
       my_func      sum  range
Gender                        
Female     Low  21014.0   63.0
Male      High   8854.9   89.0

## 三、变换和过滤
### 1. 变换函数与transform方法

**变换函数的返回值为同长度的序列**，最常用的内置变换函数是累计函数： cumcount/cumsum/cumprod/cummax/cummin ，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作。此外在 groupby 对象上还定义了填充类和滑窗类的变换函数，这些函数的一般形式将会分别在第七章和第十章中讨论，此处略过。

In [29]:
gb.cummax()
# 每个分组内的累积最大值，但展示的时候确是按每一行展示的

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0
...,...,...
195,170.2,63.0
196,170.2,63.0
197,170.2,63.0
198,193.9,89.0


pandas 中的 `cumcount()`、`cumsum()`、`cumprod()`、`cummax()`、`cummin()` 是**累积类方法**，主要用于对数据进行“逐行/逐元素累积计算”。当这些方法用于分组对象 `gb`（即 `groupby` 之后的结果）时，会**在每个分组内部独立进行累积计算**，而不是对整个数据集全局计算。


| 函数         | 含义（在分组内）                                                                 | 示例（分组内数据为 `[2, 1, 3]`） |
|--------------|----------------------------------------------------------------------------------|----------------------------------|
| `cumcount()` | 计算每个元素在分组内的**累积计数**（从 0 开始的索引）                             | 结果：`[0, 1, 2]`                |
| `cumsum()`   | 计算分组内元素的**累积和**（第 n 个元素 = 前 n 个元素之和）                        | 结果：`[2, 2+1=3, 3+3=6]`        |
| `cumprod()`  | 计算分组内元素的**累积乘积**（第 n 个元素 = 前 n 个元素之积）                      | 结果：`[2, 2×1=2, 2×3=6]`        |
| `cummax()`   | 计算分组内元素的**累积最大值**（第 n 个元素 = 前 n 个元素中的最大值）              | 结果：`[2, max(2,1)=2, max(2,3)=3]` |
| `cummin()`   | 计算分组内元素的**累积最小值**（第 n 个元素 = 前 n 个元素中的最小值）              | 结果：`[2, min(2,1)=1, min(1,3)=1]` |


In [30]:
import pandas as pd

# 示例数据：按 "group" 列分为 A、B 两组
df_tmp = pd.DataFrame({
    "group": ["A", "B", "A", "B", "B"],
    "value": [2, 1, 3, 5, 4]
})

gb_tmp = df_tmp.groupby("group")  # 按 group 分组

# 分组内累积计算
df_tmp["cumcount"] = gb_tmp["value"].cumcount()
df_tmp["cumsum"] = gb_tmp["value"].cumsum()
df_tmp["cumprod"] = gb_tmp["value"].cumprod()
df_tmp["cummax"] = gb_tmp["value"].cummax()
df_tmp["cummin"] = gb_tmp["value"].cummin()

print(df_tmp)

# 实际是按 group 列分组，计算每个组内的累计值，但展示的时候
# 是按 index 展示的



  group  value  cumcount  cumsum  cumprod  cummax  cummin
0     A      2         0       2        2       2       2
1     B      1         0       1        1       1       1
2     A      3         1       5        6       3       2
3     B      5         1       6        5       5       1
4     B      4         2      10       20       5       1


##### 练一练：
在 groupby 对象中， rank 方法也是一个实用的变换函数，请查阅它的功能并给出一个使用的例子。

In [31]:
# rank函数
# 计算数值数据沿轴的秩（1 到 n）。 默认情况下，相等的值会被分配一个秩，
# 该秩是这些值秩的平均值。
df_rank = pd.DataFrame(data={'Animal': ['cat', 'penguin', 'dog',
                                   'spider', 'snake'],
                        'Number_legs': [4, 2, 4, 8, np.nan]})
df_rank

,Animal,Number_legs
0,cat,4.0
1,penguin,2.0
2,dog,4.0
3,spider,8.0
4,snake,NaN


默认情况下，平局会被分配该组的排名平均值；也就是说b和d相等本来是2，3
但是因为默认是method='average'，所以会被分配2.5

In [32]:
s = pd.Series(range(5), index=list("abcde"))
s['d'] = s['b']
s.rank()

a    1.0
b    2.5
c    4.0
d    2.5
e    5.0
dtype: float64

以下示例显示了该方法如何使用上述参数：
- default_rank：这是不使用任何参数获得的默认行为。
- max_rank：设置 method = 'max'，具有相同值的记录使用最高排名进行排名（例如：由于“cat”和“dog”都位于第 2 和第 3 位，因此分配排名 3。）
- NA_bottom：选择 na_option = 'bottom'，如果有具有 NaN 值的记录，则它们将位于排名的底部。
- pct_rank：当设置 pct = True 时，排名表示为百分位排名。


In [33]:
df_rank['default_rank'] = df_rank['Number_legs'].rank()
df_rank['max_rank'] = df_rank['Number_legs'].rank(method='max')
df_rank['NA_bottom'] = df_rank['Number_legs'].rank(na_option='bottom')
df_rank['pct_rank'] = df_rank['Number_legs'].rank(pct=True)
df_rank


,Animal,Number_legs,default_rank,max_rank,NA_bottom,pct_rank
0,cat,4.0,2.5,3.0,2.5,0.625
1,penguin,2.0,1.0,1.0,1.0,0.250
2,dog,4.0,2.5,3.0,2.5,0.625
3,spider,8.0,4.0,4.0,4.0,1.000
4,snake,NaN,NaN,NaN,5.0,NaN


当用自定义变换时需要使用 transform 方法，**被调用的自定义函数， 其传入值为数据源的序列 ，与 agg 的传入类型是一致的，其最后的返回结果是行列索引与数据源一致的 DataFrame 。**

分组对象的 transform 方法用于**对每个分组内的所有元素应用同一个函数，且返回的结果形状与原数据一致（保持索引和行数不变），这是它与 agg（聚合为单个值）的核心区别**。

现对身高和体重进行分组标准化，即减去组均值后除以组的标准差：

In [34]:
# 对分组后的每个元素进行 “标准化” 处理，公式为 (当前值 - 分组均值) / 分组标准差，
# 也称为 “Z-score 标准化”。
gb.transform(lambda x: (x-x.mean())/x.std()).head()

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631


##### 练一练：
对于 transform 方法无法像 agg 一样，通过传入字典来对指定列使用特定的变换，如果需要在一次 transform 的调用中实现这种功能，请给出解决方案。



In [35]:
# 利用 apply 结合 DataFrame 列操作

def multi_transform(group_df):
    # 对Height列：标准化（(x-均值)/标准差）
    group_df['Height'] = (group_df['Height'] - group_df['Height'].mean()) / group_df['Height'].std()
    # 对Weight列：减去分组均值
    group_df['Weight'] = group_df['Weight'] - group_df['Weight'].mean()
    return group_df

# 删除原来的索引，Gender和level_1；不把这两个变成df的列
gb.apply(multi_transform).reset_index(drop=False)
gb.apply(multi_transform).reset_index(drop=True)

,Height,Weight
0,-0.058760,-1.918519
1,NaN,-6.918519
2,-0.236837,3.081481
3,0.653550,4.081481
4,0.534832,2.081481
...,...,...
195,0.521319,NaN
196,-0.968363,-2.759259
197,2.876435,6.240741
198,0.237570,-1.759259


前面提到了 transform 只能返回同长度的序列，但事实上还可以返回一个标量，这会使得结果被广播到其所在的整个组，这种 标量广播 的技巧在特征工程中是非常常见的。例如，构造两列新特征来分别表示样本所在性别组的身高均值和体重均值：

In [36]:
# 计算每个分组的均值
gb.transform('mean').head()

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259


### 2. 组索引与过滤
在上一章中介绍了索引的用法，那么索引和过滤有什么区别呢？

**过滤在分组中是对于组的过滤，而索引是对于行的过滤**，在第二章中的返回值，无论是布尔列表还是元素列表或者位置列表，本质上都是对于行的筛选，即如果符合筛选条件的则选入结果表，否则不选入。

**组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回 True 则会被保留， False 则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为 DataFrame 返回。**

在 groupby 对象中，**定义了 filter 方法进行组的筛选，其中自定义函数的输入参数为数据源构成的 DataFrame 本身**，在之前例子中定义的 groupby 对象中，传入的就是 df[['Height', 'Weight']] ，因此所有表方法和属性都可以在自定义函数中相应地使用，同时只需保证自定义函数的返回为布尔值即可。

例如，在原表中通过过滤得到所有容量大于100的组：

In [37]:
gb.filter(lambda x: x.shape[0] > 100)

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0
...,...,...
191,166.6,54.0
194,160.3,49.0
195,153.9,46.0
196,160.9,50.0


##### 练一练：
从概念上说，索引功能是组过滤功能的子集，请使用 filter 函数完成 loc[.] 的功能，这里假设 ” . “是元素列表。

In [38]:
gb.filter(lambda x: x.name in ['Height'])

,Height,Weight


## 四、跨列分组

### 1. apply的引入
之前几节介绍了三大分组操作，但事实上还有一种常见的分组场景，无法用前面介绍的任何一种方法处理，例如现在如下定义身体质量指数BMI：

$$
BMI = \frac{Weight}{Height^2}

$$
 
其中体重和身高的单位分别为千克和米，需要分组计算组BMI的均值。

首先，这显然不是过滤操作，因此 filter 不符合要求；其次，返回的均值是标量而不是序列，因此 transform 不符合要求；最后，似乎使用 agg 函数能够处理，但是之前强调过聚合函数是逐列处理的，而不能够 多列数据同时处理 。由此，引出了 apply 函数来解决这一问题。

### 2. apply的使用
在设计上， **apply 的自定义函数传入参数与 filter 完全一致，只不过后者只允许返回布尔值**。现如下解决上述计算问题：

In [40]:
def BMI(x):
    BMI = x['Weight'] / ((x['Height']/100) ** 2)
    return BMI.mean()

gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

除了返回标量之外， apply 方法还可以返回一维 Series 和二维 DataFrame ，但它们产生的数据框维数和多级索引的层数应当如何变化？下面举三组例子就非常容易明白结果是如何生成的：

#### 【a】标量情况：结果得到的是 Series ，索引与 agg 的结果一致

In [ ]:
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]

# x是每个分组的DataFrame

gb.apply(lambda x:0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [ ]:
# # 虽然是列表，但是作为返回值仍然看作标量
gb.apply(lambda x: [0,0])

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

#### 【b】 Series 情况：得到的是 DataFrame ，行索引与标量情况一致，列索引为 Series 的索引

In [43]:
gb.apply(lambda x: pd.Series([0,0], index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

##### 练一练：
请尝试在 apply 传入的自定义函数中，根据组的某些特征返回相同长度但索引不同的 Series ，会报错吗？

**会报错。**

原因：groupby.apply() 要求每个分组的处理结果必须保持与原分组相同的索引结构（或可对齐的索引），最终才能拼接成完整结果。

In [76]:
import pandas as pd
df_ser = pd.DataFrame({'group': ['A', 'A', 'B', 'B'], 'value': [1, 2, 3, 4]})
gb_ser = df_ser.groupby('group')

def bad_apply(s):
    # 返回相同长度但索引不同的Series（原索引为[0,1]或[2,3]，新索引为[10,11]）
    return pd.Series(s.values, index=np.arange(s.values.shape[0]))

try:
    gb_ser['value'].apply(bad_apply)
except ValueError as e:
    print("报错：", e)  # 会触发ValueError

In [78]:

def bad_apply(s):
    # 返回相同长度但索引不同的Series（原索引为[0,1]或[2,3]，新索引为[10,11]）
    # print(s.values)
    # print(np.arange(s.shape[0]).shape)

    return pd.Series(s.values, index=np.arange(s.values.shape[0]))


try:
    # gb.apply(lambda x:x)
    gb.apply(bad_apply)
except ValueError as e:
    print("报错：", e)  # 会触发ValueError
gb.apply(lambda x:x)

报错： Data must be 1-dimensional, got ndarray of shape (67, 2) instead


Height  Weight
Gender Test_Number                    
Female 1           0     158.9    46.0
                   5     158.0    51.0
                   6     162.5    52.0
                   7     161.9    50.0
                   8     163.0    48.0
...                        ...     ...
Male   3           76    174.6     NaN
                   152     NaN    62.0
                   178     NaN    76.0
                   179   175.5    73.0
                   182   170.5    73.0

[200 rows x 2 columns]

#### 【c】 DataFrame 情况：
得到的是 DataFrame ，行索引最内层在每个组原先 agg 的结果索引上，再加一层返回的 DataFrame 行索引，同时分组结果 DataFrame 的列索引和返回的 DataFrame 列索引一致。

In [52]:
gb.apply(lambda x: pd.DataFrame(np.ones((2,2)),index=['a','b'],
                                columns=pd.Index([('w','x'),('y','z')])))
gb.apply(lambda x:x)

Height  Weight
Gender Test_Number                    
Female 1           0     158.9    46.0
                   5     158.0    51.0
                   6     162.5    52.0
                   7     161.9    50.0
                   8     163.0    48.0
...                        ...     ...
Male   3           76    174.6     NaN
                   152     NaN    62.0
                   178     NaN    76.0
                   179   175.5    73.0
                   182   170.5    73.0

[200 rows x 2 columns]

##### 练一练：
请尝试在 apply 传入的自定义函数中，根据组的某些特征返回相同大小但列索引不同的 DataFrame ，会报错吗？如果只是行索引不同，会报错吗？

最后需要强调的是， **apply 函数的灵活性是以牺牲一定性能为代价换得的，除非需要使用跨列处理的分组处理，否则应当使用其他专门设计的 groupby 对象方法，否则在性能上会存在较大的差距**。同时，在使用聚合函数和变换函数时，也应当优先使用内置函数，它们经过了高度的性能优化，一般而言在速度上都会快于用自定义函数来实现。

##### 练一练：
在 groupby 对象中还定义了 cov 和 corr 函数，从概念上说也属于跨列的分组处理。请利用之前定义的 gb 对象，使用apply函数实现与 gb.cov() 同样的功能并比较它们的性能。

## 五、练习
### Ex1：汽车数据集
现有一份汽车数据集，其中 Brand, Disp., HP 分别代表汽车品牌、发动机蓄量、发动机输出。

```python

df = pd.read_csv('data/car.csv')

df.head(3)
Out[46]: 
             Brand  Price Country  Reliability  Mileage   Type  Weight  Disp.   HP
0   Eagle Summit 4   8895     USA          4.0       33  Small    2560     97  113
1  Ford Escort   4   7402     USA          2.0       33  Small    2345    114   90
2   Ford Festiva 4   6319   Korea          4.0       37  Small    1845     81   63

```

1. 先过滤出所属 Country 数超过2个的汽车，即若该汽车的 Country 在总体数据集中出现次数不超过2则剔除，再按 Country 分组计算价格均值、价格变异系数、该 Country 的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为 CoV 。

2. 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计 Price 的均值。

3. 对类型 Type 分组，对 Price 和 HP 分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。

4. 对类型 Type 分组，对 HP 进行组内的 min-max 归一化。

5. 对类型 Type 分组，计算 Disp. 与 HP 的相关系数。



### Ex2：实现transform函数
- groupby 对象的构造方法是 my_groupby(df, group_cols)

- 支持单列分组与多列分组
- 支持带有标量广播的 my_groupby(df)[col].transform(my_func) 功能

- pandas 的 transform 不能跨列计算，请支持此功能，即仍返回 Series 但 col 参数为多列

- 无需考虑性能与异常处理，只需实现上述功能，在给出测试样例的同时与 pandas 中的 transform 对比结果是否一致